In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [6]:
freq1, freq2, offset1, offset2 = np.random.rand(4, 2, 1)

In [7]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offset1, offset2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5*np.sin((time - offset1) * (freq1*10 + 10))
    series += 0.2*np.sin((time - offset2) * (freq2*20 + 20)
    )
    series += 0.1*(np.random.rand(batch_size, n_steps) - 0.5)
    return series[..., np.newaxis].astype(np.float32)

In [8]:
n_steps = 50
series = generate_time_series(10000, n_steps+1)
xtrain, ytrain = series[:7000, :n_steps], series[:7000, -1]
xvalid, yvalid = series[7000:9000, :n_steps], series[7000:9000, -1]
xtest, ytest = series[9000:, :n_steps], series[9000:, -1]

# Baseline

1. Previous Term as prediction
2. Using Linear Model (NN)

In [15]:
# Baseline MSE 
ypred = xtest[:, -1] # Predicting last value
np.mean(tf.keras.losses.mean_squared_error(ypred, ytest))

0.020552604

In [19]:
np.random.seed(42)
tf.random.set_seed(42)

model =tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = [50, 1]),
    tf.keras.layers.Dense(1),
    
])
model.compile(loss = ['mse'], optimizer = 'Adam', metrics = ['mse'])
model.fit(xtrain, ytrain, validation_data = (xvalid, yvalid), epochs = 20)

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 4s 583us/sample - loss: 0.1038 - mse: 0.1038 - val_loss: 0.0559 - val_mse: 0.0559
Epoch 2/20
7000/7000 [==============================] - 2s 327us/sample - loss: 0.0402 - mse: 0.0402 - val_loss: 0.0279 - val_mse: 0.0279
Epoch 3/20
7000/7000 [==============================] - 1s 141us/sample - loss: 0.0218 - mse: 0.0218 - val_loss: 0.0164 - val_mse: 0.0164
Epoch 4/20
7000/7000 [==============================] - 1s 131us/sample - loss: 0.0140 - mse: 0.0140 - val_loss: 0.0118 - val_mse: 0.0118
Epoch 5/20
7000/7000 [==============================] - 1s 112us/sample - loss: 0.0108 - mse: 0.0108 - val_loss: 0.0097 - val_mse: 0.0097
Epoch 6/20
7000/7000 [==============================] - 1s 111us/sample - loss: 0.0092 - mse: 0.0092 - val_loss: 0.0086 - val_mse: 0.0086
Epoch 7/20
7000/7000 [==============================] - 1s 95us/sample - loss: 0.0082 - mse: 0.0082 - val_loss: 0.0077 - val

In [21]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.evaluate(xtest, ytest)

1000/1000 [==============================] - 0s 285us/sample - loss: 0.0043 - mse: 0.0043


[0.004261311408132315, 0.004261311]

# RNNs

1. SimpleRNN : 1 layer
2. Deep RNN : 2 layer


In [16]:
model =tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(1, input_shape = [50, 1]),
])
model.compile(loss = ['mse'], optimizer = 'Adam', metrics = ['mse'])
model.fit(xtrain, ytrain, validation_data = (xvalid, yvalid), epochs = 20)

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 11s 2ms/sample - loss: 0.2888 - mse: 0.2888 - val_loss: 0.2311 - val_mse: 0.2311
Epoch 2/20
7000/7000 [==============================] - 5s 774us/sample - loss: 0.1720 - mse: 0.1720 - val_loss: 0.1204 - val_mse: 0.1204
Epoch 3/20
7000/7000 [==============================] - 4s 523us/sample - loss: 0.0830 - mse: 0.0830 - val_loss: 0.0571 - val_mse: 0.0571
Epoch 4/20
7000/7000 [==============================] - 4s 502us/sample - loss: 0.0451 - mse: 0.0451 - val_loss: 0.0376 - val_mse: 0.0376
Epoch 5/20
7000/7000 [==============================] - 4s 508us/sample - loss: 0.0349 - mse: 0.0349 - val_loss: 0.0324 - val_mse: 0.0324
Epoch 6/20
7000/7000 [==============================] - 4s 507us/sample - loss: 0.0314 - mse: 0.0314 - val_loss: 0.0298 - val_mse: 0.0298
Epoch 7/20
7000/7000 [==============================] - 3s 371us/sample - loss: 0.0290 - mse: 0.0290 - val_loss: 0.0276 - val

In [17]:
model.evaluate(xtest, ytest)

1000/1000 [==============================] - 0s 112us/sample - loss: 0.0132 - mse: 0.0132


[0.01322248201444745, 0.013222481]

In [18]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [22]:
model =tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(10, return_sequences = True, input_shape = [None, 1]),
    tf.keras.layers.SimpleRNN(1),

])
model.compile(loss = ['mse'], optimizer = 'Adam', metrics = ['mse'])
model.fit(xtrain, ytrain, validation_data = (xvalid, yvalid), epochs = 20)

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 16s 2ms/sample - loss: 0.0217 - mse: 0.0217 - val_loss: 0.0106 - val_mse: 0.0106
Epoch 2/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0091 - mse: 0.0091 - val_loss: 0.0073 - val_mse: 0.0073
Epoch 3/20
7000/7000 [==============================] - 6s 856us/sample - loss: 0.0070 - mse: 0.0070 - val_loss: 0.0060 - val_mse: 0.0060
Epoch 4/20
7000/7000 [==============================] - 6s 875us/sample - loss: 0.0060 - mse: 0.0060 - val_loss: 0.0052 - val_mse: 0.0052
Epoch 5/20
7000/7000 [==============================] - 6s 791us/sample - loss: 0.0052 - mse: 0.0052 - val_loss: 0.0047 - val_mse: 0.0047
Epoch 6/20
7000/7000 [==============================] - 6s 817us/sample - loss: 0.0048 - mse: 0.0048 - val_loss: 0.0045 - val_mse: 0.0045
Epoch 7/20
7000/7000 [==============================] - 5s 782us/sample - loss: 0.0046 - mse: 0.0046 - val_loss: 0.0042 - val_m

In [23]:
model.evaluate(xtest, ytest)

1000/1000 [==============================] - 0s 175us/sample - loss: 0.0035 - mse: 0.0035


[0.003519987504929304, 0.0035199875]

# Forecasting Several Steps Ahead

In [27]:
np.random.seed(42)
## moving x by one place and keep predicting using previously predicted values
series = generate_time_series(1, n_steps + 10)
xnew, ynew = series[:, :n_steps], series[:, n_steps:]
x = xnew
for step_ahead in range(10):
    ypred = model.predict(x[:, step_ahead:])[:, np.newaxis, :]
    x = np.concatenate([x, ypred], axis = 1)
ypred = x[:, n_steps:]

In [26]:
ypred.shape

(1, 10, 1)